In [62]:
%reset -f
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.utils import to_categorical
from wavhandler import *
from utils_train import train_test_val_split

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [63]:
data = Dataset('Wingbeats')
data.read(data='all', setting='read', loadmat=False, labels='nr')

Data: all.
Read 279566 filenames in 1.04 seconds.


In [64]:
X,y = data.filenames.tolist(), data.y.tolist()
model_name='first_' 
setting='raw'

In [65]:
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
targets = len(np.unique(y))

if setting == 'psd_dB':
    X = transform_data(X, setting=setting)

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y, random_state=0)

# # Convert label to onehot
# y_train = to_categorical(y_train, num_classes=targets)
# y_val = to_categorical(y_val, num_classes=targets)
# y_test = to_categorical(y_test, num_classes=targets)

# X_train = np.expand_dims(X_train, axis=-1)
# X_val = np.expand_dims(X_val, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

## MODEL

In [66]:
# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(5000, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(targets, activation='softmax'))

## Compile and fit

In [67]:
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]


In [68]:
import math
from utils_train import train_generator, valid_generator
target_names = list(np.unique(data.y))
batch_size = 32
model.fit_generator(train_generator(X_train, y_train, batch_size=32,
                                   target_names=target_names,
                                   setting=setting),
                   steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                   epochs=100,
                   validation_data = valid_generator(X_val, y_val,
                                                    batch_size=32,
                                                    target_names=target_names,
                                                    setting=setting),
                    validation_steps=int(math.ceil(float(len(X_test))/float(batch_size))),
                    callbacks = callbacks_list)

Epoch 1/100
 175/6291 [..............................] - ETA: 19:58 - loss: 1.0979 - acc: 0.6057

KeyboardInterrupt: 

## Test model

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)

print('loss', loss)
print('Test accuracy:', acc)

from keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + "_raw_final.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)